In [15]:
import os
import time
import xml.dom.minidom as xmldom
import math

import pandas as pd
import geopandas as gpd

In [2]:
def convert_pbf_osm (pbf_file) :
    '''
    使用osmconverter.exe将pbf转码为osm，保存在原目录
    '''
    starttime = time.time()
    base_path = 'E:/AAA_Study/202003_GraduationProject/'
    osmconvert_path = base_path + 'osmconvert'
    osm_file = pbf_file.replace('.osm.pbf','.osm')
    
    terminal_code = '{} {} > {}'.format(osmconvert_path,pbf_file,osm_file)
    os.system(terminal_code)
    endtime = time.time()
    print ('{} converted to {}'.format(pbf_file,osm_file))
    print('step total time:', round(endtime - starttime, 2),'secs')

In [62]:
def campus_extract(osmfilename):
    '''
    使用osmfilter.exe筛选全部amenity=university，保存在原目录
    '''
    starttime = time.time()
    sel_osmkey = 'amenity'
    sel_osmvalue = ['university']
    country_name = os.path.split(osmfilename)[1]
    osmfilter_path = 'E:/AAA_Study/202003_GraduationProject/' + 'osmfilter'
    
    for value in sel_osmvalue:
        sel_element = sel_osmkey + "=" + value
        print (sel_element)
        output_sel = osmfilename.replace('.osm','_{}.osm'.format(value))
        print (output_sel)
        terminal_code = '{} {} --keep="{}" -o={}'.format(osmfilter_path,osmfilename,sel_element,output_sel)
        os.system(terminal_code)
    endtime = time.time()
    print(country_name + "----------" + sel_element + " extracted success!")
    print('step total time:', round(endtime - starttime, 2),'secs')

In [95]:
def parseValidData (INPUT_OSM):
    '''
    使用xmldom对第一步提取后的osm校园文件进行数据筛选，提出building等无效数据，仅保留有地块信息的高校数据，返回sub_element_n,sub_element_w
    '''
    print('...')
    dom = xmldom.parse(INPUT_OSM)
    element = dom.documentElement
    sub_element_n = element.getElementsByTagName("node")
    sub_element_w_raw = element.getElementsByTagName("way")
    print('"way" element raw data:',len(sub_element_w_raw))
    for i in range(len(sub_element_w_raw)):
        way_tags = sub_element_w_raw[i].getElementsByTagName("tag")
        string = ''
        for j in range(len(way_tags)):   
            string = string + way_tags[j].getAttribute('k')+way_tags[j].getAttribute('v')
        if 'university' not in string or 'building' in string:
            element.removeChild(sub_element_w_raw[i])
    sub_element_w = element.getElementsByTagName("way")
    print('"way" element:',len(sub_element_w))
    print('"node" element:',len(sub_element_n))
    print('data parsing done!')
    return sub_element_n,sub_element_w